In [1]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 8689-E141

 Directory of c:\Users\jaayg\OneDrive\Escritorio\computanional_vision\c_vision_v2\segmentacion_imagenes

05/28/2025  10:27 AM    <DIR>          .
05/28/2025  10:26 AM    <DIR>          ..
05/25/2025  08:23 PM             3,536 1_segmentacion_YOLO.py
05/28/2025  10:28 AM            14,685 2_segmentation_YOLO.ipynb
               2 File(s)         18,221 bytes
               2 Dir(s)  181,856,808,960 bytes free


In [2]:
video_path = './procesamemiento_imagenes/data/store-aisle-detection.mp4'

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Open the video file
cap = cv2.VideoCapture(video_path)

#Create background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2(
    history=500,
    varThreshold=16,
    detectShadows=True
)

heatmap_refined = None

# load YOLOv11 model to segmentation
model = YOLO('yolo11n-seg')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    #Initialize the accumulator for the heatmap
    if heatmap_refined is None:
        heatmap_refined = np.zeros(frame.shape[:2], dtype=np.float32)

    # --- Step 1: Background Subtraction ---
    fgmask = bg_subtractor.apply(frame)
    # Threshold the foreground mask to create a binary mask
    _, fgmask = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)

    # --- Step 2: Segmentation with YOLO ---
    # Realizing the detection with segmentation in the whole frame
    results = model(frame, verbose=False)[0]

    #Creat an empty mask for accumulationg the segmentations from the class 'persone'
    segmentation_mask = np.zeros(frame.shape[:2], dtype=np.uint8)

    if results.masks is not None:
        # Extract the masks for the class 'person'
        masks = results.masks.data.cpu().numpy() if hasattr(results.masks, 'cpu') else results.masks.data
        classes = results.boxes.cls.cpu().numpy() if hasattr(results.boxes.cls, 'cpu') else results.boxes.cls

        for mask, cls in zip(masks, classes):
            if int(cls) == 0: # Filter for the class 'person'
                mask_bin = (mask > 0.5).astype(np.uint8) * 255
                #Resize mask_bin to the original frame size
                mask_bin_resized = cv2.resize(mask_bin, (segmentation_mask.shape[1], segmentation_mask.shape[0]), interpolation=cv2.INTER_NEAREST)
                segmentation_mask = cv2.bitwise_or(segmentation_mask, mask_bin_resized)

    # --- Paso 3: Mixing masks ---
    # Realizing an intersection between the foreground mask and the segmentation mask
    refined_mask = cv2.bitwise_and(fgmask, segmentation_mask)

    # Acumulate the refined mask into the heatmap
    heatmap_refined = cv2.add(heatmap_refined, refined_mask.astype(np.float32))

    # Intermediate visualization
    cv2.imshow("Frame Original", frame)
    cv2.imshow("Mascara Movimiento (FG)", fgmask)
    cv2.imshow("Mascara Segmentacion (Personas)", segmentation_mask)
    cv2.imshow("Mascara Refinada", refined_mask)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()


100%|██████████| 5.90M/5.90M [00:00<00:00, 34.9MB/s]
